# Un nuevo modelo SVR

## Instrucciones
Ahora que has construido un modelo SVR, construye uno nuevo con datos frescos (prueba uno de [estos conjuntos de datos de Duke](http://www2.stat.duke.edu/~mw/ts_data_sets.html)). Anota tu trabajo en un notebook. Visualiza los datos y tu modelo, y prueba su precisión usando MAPE y gráficos apropiados. También prueba ajustando distintos hiperparámetros y valores para los pasos de tiempo.